In [1]:
import sys
sys.path.append('../../../build/python')

import os # Configure which GPU
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')


import mitsuba as mi
import drjit as dr
import tensorflow as tf
import torch
%load_ext autoreload
%autoreload 2
sys.path.append('../drjit')

from interop import wrap, to_drjit, from_drjit, flatten

I0000 00:00:1728635220.949811   90485 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728635220.989228   90485 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728635220.992242   90485 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [52]:
import torch.autograd.forward_ad as fa

x_primal = torch.randn(10, 10, dtype=torch.float32)
x_tangent = torch.randn(10, 10, dtype=torch.float32)

y_primal = torch.randn(10, 10, dtype=torch.float32)
y_tangent = torch.zeros_like(y_primal)

def fn(x, y):
    return x ** 2 + y ** 2

with fa.dual_level():
    x_dual = fa.make_dual(x_primal, x_tangent)
    y_dual = fa.make_dual(y_primal, y_tangent)
    z_dual = fn(x_dual, y_dual)
    z_tangent = fa.unpack_dual(z_dual).tangent


x_primal_tf = tf.constant(x_primal.numpy())
x_tangent_tf = tf.constant(x_tangent.numpy())
y_primal_tf = tf.constant(y_primal.numpy())
y_tangent_tf = tf.constant(y_tangent.numpy())
with tf.autodiff.ForwardAccumulator(primals=[x_primal_tf, y_primal_tf],
                                    tangents=[x_tangent_tf, y_tangent_tf]) as acc:
    z_primal_tf = fn(x_primal_tf, y_primal_tf)
    z_tangent_tf = acc.jvp(z_primal_tf)


z_tangent.numpy() == z_tangent_tf.numpy()



import numpy as np
print(np.max(np.abs(z_tangent_tf - z_tangent)))
print(np.max(np.abs(z_primal_tf - z_dual)))


0.0
4.7683716e-07


In [61]:
@wrap('drjit', 'tf')
def test_fn(x, y, z):
    return x*2, y, z

x = dr.arange(dr.llvm.ad.Float, 3)
dr.enable_grad(x)

a, b, c = test_fn(x, 4, 5.0)
#assert dr.all(a == x*2) and dr.all(b == 4) and dr.all(c == 5)

a.grad = [10, 20, 30]
dr.backward_to(x)

#assert dr.all(x.grad == [20, 40, 60])

(tensor([20., 40., 60.]), 4, 5.0)


[20, 40, 60]

In [14]:
## Forward-mode AD
@wrap('drjit', 'torch')
def fn(x):
    return x**2

x = dr.arange(dr.llvm.ad.Float, 3)
dr.enable_grad(x)
y = fn(x)
y.grad = [1, 2, 1]
dr.backward_to(x)
x.grad





tensor([1., 2., 1.])


[0, 4, 4]

In [63]:
# Test for a single tensor input

@wrap('tf', 'drjit')
def dr_func(input):
    return dr.power(input, 2)

x = tf.constant([1,2,3,4], tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = dr_func(x)
grad = tape.gradient(y, x)
print("x:", x)
print("y:", y)
print("Gradient:", grad)  


x: tf.Tensor([1. 2. 3. 4.], shape=(4,), dtype=float32)
y: tf.Tensor([ 1.  4.  9. 16.], shape=(4,), dtype=float32)
Gradient: tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)


In [64]:
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [x1, x2, x3]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, vars)
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)

Result: tf.Tensor([ 3. 14. 39. 84.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [65]:
# Test with a nested list of tensors
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    inputs = sum(inputs, [])
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x4 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [[x1, x2], [x3, x4]]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, [x1, [x2, x3, x4]])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([  4.  30. 120. 340.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2., 4., 6., 8.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 3., 12., 27., 48.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([  4.,  32., 108., 256.], dtype=float32)>]


In [66]:
# Test with multiple arguments
@wrap('tf', 'drjit')
def dr_func(x1, x2):
    return dr.power(x1, 1) + dr.power(x2, 3)

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch([x1, x2])
    result = dr_func(x1, x2)
grad = tape.gradient(result, [x1, x2])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([ 2. 10. 30. 68.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [67]:
# Test with keyword argument
@wrap('tf', 'drjit')
def dr_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += dr.power(x, i+1)

    for i, x in enumerate(kwargs.values()):
        result += 4*x

    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 4, 5], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch([x1, x2, x3])
    result = dr_func([x1, x2], x3=x3)
grad = tape.gradient(result, [x1, x2, x3])
print(grad)

[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 4.,  6.,  8., 10.], dtype=float32)>, None]


In [70]:
# Test with dict argument
@wrap('tf', 'drjit')
def dr_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += dr.power(x, i+1)

    for i, x in enumerate(args[1].values()):
        result += 4*(i+1)*x

    return result

def tf_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += tf.pow(x, i+1)

    for i, x in enumerate(args[1].values()):
        result += 4*(i+1)*x

    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 4, 5], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch([x1, x2, x3])
    result = dr_func(x1, {'x2' : x2, 'x3': x3})
grad = tape.gradient(result, [x2])
print(result)
for g in grad:
    print(g)

tf.Tensor([320. 332. 336. 348.], shape=(4,), dtype=float32)
tf.Tensor([4. 4. 4. 4.], shape=(4,), dtype=float32)


In [76]:
from tensorflow.autodiff import ForwardAccumulator

In [69]:
# Test with a non differentiable input
@wrap('tf', 'drjit')
def dr_func(x1, x2):
    return dr.power(x1, 1) + dr.power(x2, 1)

def tf_func(x1, x2):
    return tf.pow(x1, 1) + tf.cast(tf.pow(x2, 1), x1.dtype)

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.int32)
with tf.GradientTape() as tape:
    tape.watch([x1, x2])
    result = dr_func(x1, x2)
grad = tape.gradient(result, [x1, x2])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
None


In [6]:
import tensorflow as tf
tf.constant([1,2,3,4], tf.float32)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>

In [17]:
import drjit as dr
from drjit.cuda.ad import Float
from drjit.cuda.ad import Int32

t = Int32
is_diff = False

@wrap('drjit', 'torch')
def test_fn(x):
    return x * 2

x = dr.arange(t, 3)
dr.enable_grad(x)
y = test_fn(x)
assert dr.all(y == [0, 2, 4])


if is_diff:
    y.grad = [10, 20, 30]
    dr.backward_to(x)
    assert dr.all(x.grad == [20, 40, 60])
else:
    assert not dr.grad_enabled(y)






In [35]:
from interop import from_drjit, to_drjit

t = dr.llvm.ad.Float
x = dr.arange(t, 3)
z, _ = from_drjit(x, 'tf', True)
y = z*2
z_ = to_drjit(y, 'tf')


In [12]:
import drjit as dr


@wrap('drjit', 'tf')
def test_fn(x):
    y = x*2
    print(y.device)
    return y

t = dr.cuda.ad.Float
x = dr.arange(t, 3)
y = test_fn(x)



/job:localhost/replica:0/task:0/device:GPU:0
